# 构造数据集
## 相差处理
处理各个初始文件，3900为一组，后一组减去前一组，得到的数据的后3800个可视为有效信号，予以保留。
首先明确各个文件名

In [7]:
import os
import numpy as np
import matplotlib.pyplot as plt

os.listdir('./')

['zhuhai.h5',
 'hand-data-2019-12-04-09-11-55.txt',
 'foot_data.txt',
 'stick-data-2019-12-04-09-18-45.txt',
 'train_label.txt',
 'normal-data-2019-12-04-09-06-26.txt',
 'train_data.txt',
 'wave_classify.ipynb',
 'stick-data-2019-12-04-09-04-41.txt',
 'normal-data-2019-12-04-09-21-09.txt',
 'normal_data.txt',
 'test_data.txt',
 'stick_data.txt',
 'foot-data-2019-12-04-08-53-22.txt',
 'dataset',
 'fangyan.h5',
 'hand_data.txt',
 'hand-data-2019-12-04-09-19-55.txt',
 'val_data.txt',
 'train.txt',
 'test.txt',
 'val.txt',
 'test_label.txt',
 'val_label.txt',
 'foot-data-2019-12-04-09-17-32.txt',
 'wave_dataset.ipynb',
 '.ipynb_checkpoints',
 'DVS.h5']

In [8]:
def loadData(data):
    '''
    相差法处理原始数据,得到有效数据
    '''
    data = np.array(data)
    valid_data = []
    for i in range(0,len(data)-3989,3900):
        temp = [x for x in map(abs, data[i+3900:i+3900*2]-data[i:i+3900])]
        valid_data.append(temp[100:])
    print('共{}组数据，每组数据有{}个点'.format(len(valid_data),len(valid_data[0])))
    return valid_data

In [14]:
# 处理脚踏数据
data = []
with open('foot-data-2019-12-04-08-53-22.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))
with open('foot-data-2019-12-04-09-17-32.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))

foot_data = loadData(data)    
# 保存脚踏数据
with open('foot_data.txt','w') as f:
    for line in foot_data:
        f.write(str(line)+'\n')

共499组数据，每组数据有3800个点


In [18]:
# 处理手触数据
data = []
with open('hand-data-2019-12-04-09-11-55.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))
with open('hand-data-2019-12-04-09-19-55.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))

hand_data = loadData(data)
# 保存手触数据
with open('hand_data.txt','w') as f:
    for line in hand_data:
        f.write(str(line)+'\n')

共499组数据，每组数据有3800个点


In [21]:
# 处理stick数据
data = []
with open('stick-data-2019-12-04-09-04-41.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))
with open('stick-data-2019-12-04-09-18-45.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))

stick_data = loadData(data)
# 保存手触数据
with open('stick_data.txt','w') as f:
    for line in stick_data:
        f.write(str(line)+'\n')

共499组数据，每组数据有3800个点


In [23]:
# 处理normal数据
data = []
with open('normal-data-2019-12-04-09-06-26.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))
with open('normal-data-2019-12-04-09-21-09.txt','r') as f:
    for line in f:
        data.append(int(line.strip()))

normal_data = loadData(data)
# 保存手触数据
with open('normal_data.txt','w') as f:
    for line in normal_data:
        f.write(str(line)+'\n')

共499组数据，每组数据有3800个点


In [6]:
plt.figure(figsize=(12,8))
plt.subplot(221)
plt.title('foot_data')
plt.plot(foot_data[1])
plt.subplot(222)
plt.title('hand_data')
plt.plot(hand_data[1])
plt.subplot(223)
plt.title('stick_data')
plt.plot(stick_data[1])
plt.subplot(224)
plt.title('normal_data')
plt.plot(normal_data[11])
plt.show()

NameError: name 'plt' is not defined

## 划分数据集
每类数据共499个，则训练数据417个，验证数据41个，测试数据41个。\
建立`train.txt`, `train_label.txt`, `val.txt`, `val_label.txt`, `test.txt`, `test_label.txt`。
1. 依次取出四种数据，打乱内部顺序
2. 取前417个放入`train_data`，接着41个放入`val_data`，最后41个放入`test_data`。
3. 构造依次417个foot，hand，stick，normal的`train_label`，同样的还有`val_label`,`test_label`
4. 利用`sklearn.utils.shuffle()`函数同步打乱data与其label并放入各自文件夹

In [1]:
from sklearn.utils import shuffle

In [2]:
def handle_data(name,data,train_data,val_data,test_data,train_label,val_label,test_label):
    '''
    拿出数据，打乱，存放
    '''
    data = shuffle(data)
    for i in range(417):
        train_data.append(data[i])
    for i in range(417,417+41):
        val_data.append(data[i])
    for i in range(417+41,499):
        test_data.append(data[i])
        
    # 将label编码为整数，normal-0，foot-1，hand-2,stick-3
    if name == 'normal': name = 0
    elif name == 'foot': name = 1
    elif name == 'hand': name = 2
    else: name = 3
    for i in range(417):
        train_label.append(name)
    for i in range(41):
        val_label.append(name)
        test_label.append(name)
    return train_data,val_data,test_data,train_label,val_label,test_label

In [3]:
file_name = ['foot','hand','stick','normal']
train_data,val_data,test_data,train_label,val_label,test_label = [],[],[],[],[],[]
for name in file_name:
    data = []
    with open(name+'_data.txt','r') as f:
        for line in f:
            data.append(line.strip('[]\n'))
    handle_data(name,data,train_data,val_data,test_data,train_label,val_label,test_label)
    print('当前有{}条训练数据，{}条验证数据，{}条测试数据'.format(len(train_data),len(val_data),len(test_data)))
    print('当前有{}个训练标签，{}个验证标签，{}个测试标签'.format(len(train_label),len(val_label),len(test_label)))#

当前有417条训练数据，41条验证数据，41条测试数据
当前有417个训练标签，41个验证标签，41个测试标签
当前有834条训练数据，82条验证数据，82条测试数据
当前有834个训练标签，82个验证标签，82个测试标签
当前有1251条训练数据，123条验证数据，123条测试数据
当前有1251个训练标签，123个验证标签，123个测试标签
当前有1668条训练数据，164条验证数据，164条测试数据
当前有1668个训练标签，164个验证标签，164个测试标签


In [5]:
# 成对打乱数据
train_data, train_label = shuffle(train_data, train_label)
val_data, val_label = shuffle(val_data, val_label)
test_data, test_label = shuffle(test_data, test_label)
# 写入文件
with open('train_data.txt','w') as f:
    for line in train_data:
        f.write(line+'\n')
with open('val_data.txt','w') as f:
    for line in val_data:
        f.write(line+'\n')
with open('test_data.txt','w') as f:
    for line in test_data:
        f.write(line+'\n')
with open('train_label.txt','w') as f:
    for line in train_label:
        f.write(str(line)+'\n')
with open('val_label.txt','w') as f:
    for line in val_label:
        f.write(str(line)+'\n')
with open('test_label.txt','w') as f:
    for line in test_label:
        f.write(str(line)+'\n')